# Import des modules


In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from collections import Counter
from sklearn.pipeline import Pipeline

#Selection
from sklearn.model_selection import train_test_split, GridSearchCV, cross_validate, KFold

from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, mean_absolute_percentage_error
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MultiLabelBinarizer, MinMaxScaler

#Modèles
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier

#Metriques
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, classification_report


In [16]:
fc = pd.read_csv('fc_after_feature_engineering.csv')
print(fc.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 39 columns):
 #   Column                                     Non-Null Count  Dtype  
---  ------                                     --------------  -----  
 0   a_quitte_l_entreprise                      1470 non-null   bool   
 1   age                                        1470 non-null   float64
 2   annee_experience_totale                    1470 non-null   int64  
 3   annees_dans_l_entreprise                   1470 non-null   int64  
 4   annees_dans_le_poste_actuel                1470 non-null   int64  
 5   annees_depuis_la_derniere_promotion        1470 non-null   int64  
 6   annes_sous_responsable_actuel              1470 non-null   int64  
 7   augementation_salaire_precedente           1470 non-null   int64  
 8   distance_domicile_travail                  1470 non-null   int64  
 9   domaine_etude_Entrepreunariat              1470 non-null   float64
 10  domaine_etude_Infra & Cl

# Séparation train test simple
- Des métriques d’évaluation calculées pour chaque modèle, sur le jeu d’apprentissage et le jeu de test.

In [17]:
columns_base = ['age', 'annees_dans_l_entreprise', 'distance_domicile_travail', 'genre', 'heure_supplementaires','satisfaction_globale','statut_marital','revenu_mensuel']
columns_domaine_etude = [col for col in fc.columns if col.startswith('domaine_etude')]
columns_poste = [col for col in fc.columns if col.startswith('poste')]

all_columns = columns_base + columns_domaine_etude + columns_poste

X = fc[all_columns]
y = fc['a_quitte_l_entreprise']

X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80, random_state=666)

# Validation croisée simple
- cross_validate

In [18]:
def perform_cross_validation(
    X: pd.DataFrame,
    y: pd.Series,
    model,
    cross_val_type, # La variante de validation croisée que nous souhaitons utiliser
    scoring_metrics: tuple, # Metriques de notre choix
    return_estimator=False, # Si nous souhaitons stocker les modèles de chaque fold
    groups=None, # Nous verrons l’utilité de cet argument juste après
):
    scores = cross_validate(
        model,
        X.to_numpy(),
        y.to_numpy(),
        cv=cross_val_type,
        return_train_score=True,
        return_estimator=return_estimator,
        scoring=scoring_metrics,
        groups=groups,
    )

    for metric in scoring_metrics:
        # la moyenne des scores (performance moyenne du modèle)
        print(
            "{metric} Train Average : {metric_value}".format(
                metric=metric,
                metric_value=round(np.mean(scores["train_" + metric]),2),
            )
        )
        # la standard deviation des scores (stabilité/variance du modèle)
        print(
            "{metric} Train Standard Deviation : {metric_value}".format(
                metric=metric, metric_value=round(np.std(scores["train_" + metric]),2)
            )
        )
        print(
            "{metric} Test Average : {metric_value}".format(
                metric=metric, metric_value=round(np.mean(scores["test_" + metric]),2)
            )
        )
        print(
            "{metric} Test Standard Deviation : {metric_value}".format(
                metric=metric, metric_value=round(np.std(scores["test_" + metric]),2)
            )
        )
        print("------")

    return scores

# Fonctions

In [19]:
list_model = []

def perform_model_class(model_name, model, X_train, y_train, X_test, y_test):

        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)

        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, average="macro")
        #F1-score : compromis entre précision et rappel.
        #average="macro" : moyenne simple entre classes (équilibre toutes les classes, même si elles sont rares).
        #average="weighted" : pondérée par le nombre d’exemples par classe.
        #average="micro" : global (utilise les VP/FP/FN de toutes les classes confondues).

        model_results = {
                'Model': model_name,
                'Accuracy': accuracy,
                'F1': f1
            }

        # Matrice de confusion
        print("Matrice de confusion :")
        print(confusion_matrix(y_test, y_pred))
        # Rapport complet (precision, recall, f1, support)
        print("\nRapport de classification :")
        print(classification_report(y_test, y_pred))

        list_model.append(model_results)

# Modele DUMMY
- DummyClassifier

In [20]:
# Dummy Regressor (baseline)
print('############### MODELE DummyClassifier ################\n')
model_name = 'DummyClassifier'
model = DummyClassifier(strategy="most_frequent")
perform_model_class(model_name, model, X_train, y_train, X_test, y_test)

print(list_model)

###################################

classification_scoring_metrics = ("accuracy", "f1_macro")

scores_DummyClassifier = perform_cross_validation(
        X=X,
        y=y,
        model=DummyClassifier(),
        cross_val_type=KFold(n_splits=5, shuffle=True, random_state=666), #Par défaut, le nombre de folds est 5
        scoring_metrics=classification_scoring_metrics,
    )



############### MODELE DummyClassifier ################

Matrice de confusion :
[[252   0]
 [ 42   0]]

Rapport de classification :
              precision    recall  f1-score   support

       False       0.86      1.00      0.92       252
        True       0.00      0.00      0.00        42

    accuracy                           0.86       294
   macro avg       0.43      0.50      0.46       294
weighted avg       0.73      0.86      0.79       294

[{'Model': 'DummyClassifier', 'Accuracy': 0.8571428571428571, 'F1': 0.46153846153846156}]
accuracy Train Average : 0.84
accuracy Train Standard Deviation : 0.0
accuracy Test Average : 0.84
accuracy Test Standard Deviation : 0.02
------
f1_macro Train Average : 0.46
f1_macro Train Standard Deviation : 0.0
f1_macro Test Average : 0.46
f1_macro Test Standard Deviation : 0.01
------


/Users/tico/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tico/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/tico/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


 # Modele LINEAIRE

# Modele NON LINEAIRE
- RandomForest, XGBoost ou CatBoost
- Métriques d’évaluation en classification : matrice de confusion, rappel et précision.
- Scores (présence d’overfit ou non, capacité d’éviter les faux positifs ou faux négatifs)

# Amélioration de la classification
- demandez-vous si éviter des faux positifs est plus important qu’éviter des faux négatifs.